# Структура анализа
### 1. Сбор данных
### 2. Подготовка данных для анализа
### 3. Анализ

<font color=red>Не запускайте код в 1-2 разделах. Он представлен в качестве мета кода, для демонстрации концептов или для приведения примеров</font>


# 1. Сбор данных

### API
После экспериментов с API Comtrade выявлена оптимальная структура запросов:
- Докладчики - индивидуальный код
- Партнеры - все
- Календарное значение агрегации - год
- Код экономической деятельности - 4
    - 6-значные коды дают результаты, превышающие лимиты API
    - 2-значные коды не содержат полные данные по объему поставки
- Максимальное количество записей - 100к
    - Выходные данные по таким странам как Германия составили 54к наблюдений по всем группам товаров за 2017 год.
    - Все остальные страны скорее всего не привысят лимит.
- Все наименования товаров

### SQL - AWS PostgreSQL 
#### доступ к базе данных - в будущем по требованию

- Создана облачная база данных в которую записываются результаты API запросов.
- Таким образом не является необходимым делать новые API запросы на повторяющиеся условия анализа (одинаковые 
    календарные периоды, страны и т.д.)

### Данные в свободном доступе 
Следующие таблицы представлены Comtrade 

###### Коды наименований товаров

In [ ]:
comtrade=> select * from commodity_codes_en limit 5;
  id  |                     commodity_name                     
------+--------------------------------------------------------
 0402 | Milk and cream, concentrated or containing added sugar
 0403 | Buttermilk, curdled milk and cream or yogurt
 0406 | Cheese and curd.
 0407 | Birds' eggs, in shell, fresh, preserved or cooked.
 0409 | Natural honey.

In [ ]:
comtrade=> select count(*) from commodity_codes_en;
 count 
-------
  1227

###### Торговые партнеры

In [351]:
comtrade=> select * from partners limit 5;
 id  |       partner_name       
-----+--------------------------
   0 | World
   4 | Afghanistan
 472 | Africa CAMEU region, nes
   8 | Albania
  12 | Algeria

SyntaxError: invalid syntax (<ipython-input-351-8bbffbb87795>, line 1)

In [ ]:
comtrade=> select count(*) from partners;
 count 
-------
   292

###### Другие таблицы
Также добавлены таблицы докладывающих стран, торговых режимов и русский перевод 
наименований товаров с группами товаров.

Кроме того создана свобдная таблица отчетности на данный момент содержащая 160к рядов.
Каждый ряд - доклад страны по импорту конкретного наименования (4-значный код) за отчетный год.


In [ ]:
comtrade=> select * from commodity_codes_ru limit 5;
   id   |                  commodity_name                  |      product_group      
--------+--------------------------------------------------+-------------------------
 01     | Живые животные                                   | прочая сельхозпродукция
 0101   | Лошади, ослы и их помеси                         | прочая сельхозпродукция
 010110 | Чистопородные племенные лошади, ослы и их помеси | прочая сельхозпродукция
 010121 | Чистопородные племенные лошади                   | прочая сельхозпродукция
 010129 | Лошади обычные                                   | прочая сельхозпродукция

In [ ]:
comtrade=> select count(*) from trade_aggregation_entries;
 count  
--------
 163137

###### Обобщение составленных таблиц

In [ ]:
comtrade=> SELECT                                  
   relname as "Table",
   pg_size_pretty(pg_total_relation_size(relid)) As "Size",
   pg_size_pretty(pg_total_relation_size(relid) - pg_relation_size(relid)) as "External Size"
   FROM pg_catalog.pg_statio_user_tables ORDER BY pg_total_relation_size(relid) DESC;
           Table           |  Size   | External Size 
---------------------------+---------+---------------
 trade_aggregation_entries | 34 MB   | 15 MB
 commodity_codes_ru        | 1256 kB | 216 kB
 commodity_codes_en        | 264 kB  | 88 kB
 reporters                 | 80 kB   | 64 kB
 partners                  | 80 kB   | 64 kB
 quantity_codes            | 48 kB   | 40 kB
 trade_regimes             | 48 kB   | 40 kB
 partner_iso               | 24 kB   | 24 kB
 reporter_iso              | 24 kB   | 24 kB

In [ ]:
### Следующие шаги

Выявлен существенный bottleneck добавления новых записей в таблицы SQL на уровне Pandas.
Не смотря на то, что не использованы петли (loops), а apply - задержки в добавлении записей существенны для SQL.

## Недостаточно данных

Из-за указанной проблемы удалось собрать только данные по трем странам, не входящим в Европейский регион.
С другой стороны - создана система API запросов, автоматически направляющих данные в SQL базу данных для 
долгосрочного использования.

Возможно рассмотрение хранения полученных по API запросам данных для упрощения задачи в будущем.

<font color=red> ## Задание выполнено лишь в части </font>

# 2. Подготовка данных для анализа

### Загрузка библиотек
#### - Основные библиотеки для работы с данными (время, данные, запросы)
#### - Загрузка, чтение данных и файловая навигация
#### - SQL Alchemy - работа с базами данных

In [1]:
import datetime
import pandas as pd
import requests
import time

import json
import os

from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Enum, ForeignKey, BigInteger
from sqlalchemy.dialects.postgresql import TEXT, VARCHAR, INTEGER
from sqlalchemy.dialects.postgresql import TIMESTAMP, ARRAY
from sqlalchemy import Boolean
from sqlalchemy import desc
from sqlalchemy import UniqueConstraint
from sqlalchemy.sql import select
from sqlalchemy.sql import exists

import pkg_resources

### Загрузка данных в таблицу
Запуск этой ячейки не даст результатов, т.к. модуль **credentials** исключен из начтоящего репозитория. В нем содержатся пароли к личным облачным сервисам, на одном из которых расположена база данных

In [2]:
json_path = pkg_resources.resource_filename('credentials', 'passwords.json')
data = json.load(open(json_path))
dataslap_postgres = data["aws"]["personal"]["dataslap"]["postgres"]["free_20gb"]["dataslap_user"]
engine_test = create_engine('postgres://%(username)s:%(password)s@%(host)s:%(port)s/comtrade' %
                            {"username": dataslap_postgres["username"],
                             "password": dataslap_postgres["password"],
                             "host": dataslap_postgres["host"],
                             "port": dataslap_postgres["port"]})
Base_item = declarative_base()

### Классы SQL таблиц
**SQL Alchemy** использует ORM для декларирования структуры таблиц.
Ниже представлены структуры таблиц используемых для анализа, остальные таблицы исключены. 
Базовая информация о таблицах была представлена ранее.
У каждого класса (таблицы) стандартная 3NF структура, в большинстве случаев у таблицы два поля - id (первичный ключ) и основное поле. 
**Id** в большинстве случаев взяты напрямую из API Comtrade для облегчения объединения таблиц и упрощения сбора данных - таким образом сырые данные API используются для добавления данных в таблицы без существенных изменений.

**Id** представлены в формате числа кроме кодов товаров - так как там присутствует ноль в начале кода.

**Trade_aggregation_entry** - основная сводная таблица с внешними ключами в качестве привязок к другим таблицам. Bigint использован для отображения численных торговых агрегаций.

In [4]:
class Reporter(Base_item):
    __tablename__ = "reporters"
    id = Column(INTEGER, primary_key=True)
    reporter_name = Column(TEXT, unique=True)

    def __repr__(self):
        return "<Reporter(id='%s', reporter_name='%s')>"\
            % (self.id, self.reporter_name)

class Partner(Base_item):
    __tablename__ = "partners"
    id = Column(INTEGER, primary_key=True)
    partner_name = Column(TEXT, unique=True)

    def __repr__(self):
        return "<Partner(id='%s', partner_name='%s')>"\
            % (self.id, self.partner_name)
    
class Trade_regimes(Base_item):
    __tablename__ = "trade_regimes"
    id = Column(INTEGER, primary_key=True)
    trade_regime_name = Column(TEXT, unique=True)

    def __repr__(self):
        return "<Trade_regime(id='%s', trade_regime_name='%s')>"\
            % (self.id, self.trade_regime_name)
    
class Commodity_code_RU(Base_item):
    __tablename__ = "commodity_codes_ru"
    id = Column(String(6), primary_key=True)
    commodity_name = Column(TEXT, unique=False)
    product_group = Column(TEXT, unique=False)

    def __repr__(self):
        return "<Commodity_code_RU(id='%s', commodity_name='%s', product_group='%s')>"\
            % (self.id, self.commodity_name, self.product_group)
    
    
class Quantity_code(Base_item):
    __tablename__ = "quantity_codes"
    id = Column(INTEGER, primary_key=True)
    quantity_name = Column(TEXT, unique=True)

    def __repr__(self):
        return "<Quantity_code(id='%s', quantity_name='%s')>"\
            % (self.id, self.quantity_name)
    
class Trade_aggregation_entry(Base_item):
    __tablename__ = "trade_aggregation_entries"
    id = Column(BigInteger, primary_key=True)

    pf_code = Column(String(6), unique=False)

    year = Column(Integer)
    period = Column(Integer)
    period_description = Column(TEXT)

    aggregation_level = Column(Integer)

    trade_regime = Column(INTEGER, ForeignKey("trade_regimes.id"), nullable=False)
    reporter = Column(INTEGER, ForeignKey("reporters.id"), nullable=False)
    partner = Column(INTEGER, ForeignKey("partners.id"), nullable=False)
    commodity = Column(String(6), ForeignKey("commodity_codes_en.id"), nullable=False)
    quantity_code = Column(INTEGER, ForeignKey("quantity_codes.id"), nullable=False)

    trade_quantity = Column(BigInteger)
    net_weight = Column(BigInteger)
    trade_value = Column(BigInteger)

    __table_args__ = (UniqueConstraint('period', 'trade_regime',
                                        'reporter', 'partner',
                                        'commodity',name='_trade_entry'),
                     )


    def __repr__(self):
        return "<Trade_aggregation_entry(id='%s', pf_code='%s', year='%s', period='%s',\
                    period_description='%s', aggregation_level='%s', trade_regime='%s',\
                    reporter='%s', partner='%s', commodity='%s',\
                    quantity_code='%s', trade_quantity='%s',\
                    net_weight='%s', trade_value='%s')>"\
        % (self.id, self.pf_code, self.year, self.period, self.period_description, self.aggregation_level,
            self.trade_regime, self.reporter, self.partner, self.commodity,
            self.quantity_code, self.trade_quantity, self.net_weight, self.trade_value)

### Создание сессии для SQL запросов

In [190]:
Base_item.metadata.create_all(engine_test)
Session_test = sessionmaker(bind=engine_test)
session_test = Session_test()

### Основной запрос и объединение (join) таблиц

In [191]:
q = (session_test.query(Trade_aggregation_entry, Trade_regimes, Reporter, Partner, Commodity_code_RU, Quantity_code)
    .join(Trade_regimes, Trade_aggregation_entry.trade_regime == Trade_regimes.id)
    .join(Reporter, Trade_aggregation_entry.reporter == Reporter.id)
    .join(Partner, Trade_aggregation_entry.partner == Partner.id)
    .join(Commodity_code_RU, Trade_aggregation_entry.commodity == Commodity_code_RU.id)
    .join(Quantity_code, Trade_aggregation_entry.quantity_code == Quantity_code.id).all())

### Функция перевода данных запросов в словари Python

In [15]:
def inst_to_dict(inst, delete_id=True):
    dat = {}
    for column in inst.__table__.columns:
        dat[column.name] = getattr(inst, column.name)
    # if delete_id:
    #     dat.pop("num")
    return dat

### Сведение в финальную таблицу для анализа и запись в csv таблицу
Отброшены столбцы содержащие **id** из соединенных таблиц.
Дополнительно задан порядок столбцов для лучшего визуального восприятия.

In [192]:
aggregations = pd.DataFrame([inst_to_dict(i[0]) for i in q])
regimes = pd.DataFrame([inst_to_dict(i[1]) for i in q]).drop("id", axis = 1)
reporters = pd.DataFrame([inst_to_dict(i[2]) for i in q]).drop("id", axis = 1)
partners = pd.DataFrame([inst_to_dict(i[3]) for i in q]).drop("id", axis = 1)
commodities = pd.DataFrame([inst_to_dict(i[4]) for i in q]).rename(columns={'id':'ТНВЭД'})
quantities = pd.DataFrame([inst_to_dict(i[5]) for i in q]).drop("id", axis = 1)

final_data = pd.concat([aggregations, regimes, reporters, partners, commodities, quantities], axis = 1)

final_data = final_data.drop(["aggregation_level", "id", "period_description", 
                 "pf_code", "quantity_code", "reporter", "trade_regime", "period", "partner", "commodity"], axis = 1)

final_data = final_data.loc[:,["year", "reporter_name",  "partner_name", "trade_regime_name",
           "ТНВЭД", "commodity_name", "product_group", "quantity_name",
          "trade_quantity", "net_weight", "trade_value"]]

### Фильтрация торгового партнера "World"
**World** содержит общую сумму торговых операций от всех партнеров. Так как в данном анализе будет рассматриваться импортная деятельность всех торговых партнеров, данные с **World** не представляют ценности для данного анализа.

Фильтрованная таблица записана на диск и используется для основного анализа.

In [193]:
final_data.loc[final_data["partner_name"] != "World",:].to_csv("final_comtrade.csv", index = False)

# 3. Анализ

#### Загрузка базы данных, сформированной в предыдущих секциях

In [194]:
read_final = pd.read_csv("final_comtrade.csv", index_col = False, dtype={'ТНВЭД': object})

In [195]:
read_final.head()

,year,reporter_name,partner_name,trade_regime_name,ТНВЭД,commodity_name,product_group,quantity_name,trade_quantity,net_weight,trade_value
0,2016,Afghanistan,Belgium,Import,0402,Молоко и сливки концентрированные,готовое продовольствие,Weight in kilograms,243459,243459,254254
1,2016,Afghanistan,China,Import,0402,Молоко и сливки концентрированные,готовое продовольствие,Weight in kilograms,159285,159285,110752
2,2016,Afghanistan,Denmark,Import,0402,Молоко и сливки концентрированные,готовое продовольствие,Weight in kilograms,77958,77958,91419
3,2016,Afghanistan,France,Import,0402,Молоко и сливки концентрированные,готовое продовольствие,Weight in kilograms,2233348,2233348,1934260
4,2016,Afghanistan,Iran,Import,0402,Молоко и сливки концентрированные,готовое продовольствие,Weight in kilograms,2538333,2538333,1774192


#### Выбор стран для анализа
Так как были загружены только 3 страны (частично), анализ будет проведен на примере одной страны , группе из двух стран и всех стран как "синтетического" региона для демонстрации. Такая группировка является абсолютно искуственной.

Были загружены следующие страны: Афганистан, Албания, Алжир

In [197]:
read_final["reporter_name"].unique().tolist()

['Afghanistan', 'Albania', 'Algeria']

#### Объем импорта всех наименований товаров по характерному количеству
Каждое наименование товаров измеряется в соответствующей системе. Из отраженных в данный момент в базе данных измерений представленны следующие:

In [242]:
read_final["quantity_name"].unique().tolist()

['Weight in kilograms',
 'No Quantity',
 'Volume in litres',
 'Number of items',
 'Number of pairs',
 'Electrical energy in thousands of kilowatt-hours',
 'Volume in cubic meters',
 'Area in square metres',
 'Length in metres',
 'Weight in carats']

###### Данные фильтруются на "No quantity"

In [295]:
import_volume_slice_num = read_final.loc[read_final.quantity_name != "No Quantity",["year", "reporter_name", 
                                                                         "ТНВЭД", "commodity_name", 
                                                                        "quantity_name", "trade_quantity"]]


trade_quantity_aggr =\
pd.DataFrame(import_volume_slice_num.\
            loc[import_volume_slice_num["quantity_name"] != "Quantity",:].\
            groupby(["quantity_name", "ТНВЭД"])["trade_quantity"].\
            sum()).\
            join(trade_code_labels).\
            set_index('trade_quantity', append=True).\
            sort_index(level = [0, 2], ascending=False).reset_index(level=2)

trade_quantity_aggr.columns = ["Количество импорта", "Наименование товара"]

trade_quantity_aggr.to_csv("Агрегация товаров по количеству")

trade_quantity_aggr

Количество импорта  \
quantity_name                                    ТНВЭД                       
Weight in kilograms                              6801          18655534548   
                                                 2710          10930507497   
                                                 1101           9955236978   
                                                 1001           9734437783   
                                                 2703           8273386931   
                                                 2523           5829368839   
                                                 1005           4513146127   
                                                 1701           3652969821   
                                                 2711           2311975358   
                                                 7207           1878572641   
                                                 7214           1807943905   
                                                 7204           1280019841   
                                                 2304           1264654214   
                                                 2713           1232574911   
                                                 1518           1211587824   
                                                 1507            745008860   
                                                 7216            681795948   
                                                 2704            651929501   
                                                 7208            632718533   
                                                 2701            630109927   
                                                 0402            560998234   
                                                 1003            550617537   
                                                 0902            493147090   
                                                 7308            469917303   
                                                 0803            462308110   
                                                 2517            447665157   
                                                 7304            444841935   
                                                 2515            412959803   
                                                 2836            403592582   
                                                 0713            381562235   
...                                                                    ...   
Number of items                                  8901                   13   
                                                 6210                   13   
                                                 9404                   12   
                                                 8453                   11   
                                                 9204                    7   
                                                 8451                    6   
                                                 9112                    4   
                                                 8455                    4   
                                                 9110                    2   
                                                 8904                    2   
                                                 8434                    2   
                                                 8410                    2   
                                                 8486                    1   
                                                 8441                    0   
                                                 8413                    0   
Length in metres                                 3706               219340   
Electrical energy in thousands of kilowatt-hours 2716             14779201   
Area in square metres                            6908             75325476   
                                                 7005             13257365   


### Объем импорта за 2016-2017 год
Определяем интересующий временной период и столбцы, испольщуемые для вычислений.
Дополнительно создаем таблицу расшифровки кодов ТНВЭД из эстетических соображений визуализации таблицы.

In [223]:
import_volume_slice = read_final.loc[read_final.year.isin([2016, 2017]),["year", "reporter_name", 
                                                                         "ТНВЭД", "commodity_name",
                                                                        "product_group", "trade_value"]]

trade_code_labels = read_final.loc[:,["ТНВЭД", "commodity_name", "product_group"]].drop_duplicates().fillna("")
trade_code_labels.index = trade_code_labels["ТНВЭД"]
trade_code_labels = trade_code_labels.drop("ТНВЭД", axis = 1)

#### Объем импорта за 2016-2017 г. в Афганистане по наименованию товара

In [229]:
one_country_total_value_import =\
            pd.DataFrame(import_volume_slice.\
            loc[import_volume_slice["reporter_name"] == "Afghanistan",:].\
            groupby("ТНВЭД")["trade_value"].\
            sum().\
            sort_values(ascending = False)).\
            join(trade_code_labels, how = "left")

one_country_total_value_import.columns = ["Объем импорта, в ден. выражении", "Наименование товара", "Группа товаров"]
one_country_total_value_import.head()

,"Объем импорта, в ден. выражении",Наименование товара,Группа товаров
ТНВЭД,,,
9999,2342430530,Неклассифицированные товары,
1101,664062344,Пшеничная мука,зерно и продукты перемола
2703,598169115,Торф,
5808,334519589,"Тесьма, отделочные материалы в куске и схожие ...",текстильные материалы и изделия
1518,316489768,Непищевые модифицированные жиры и масла,химические вещества и материалы


#### Объем импорта за 2016-2017 г. в Афганистане по группе товара

In [297]:
one_country_total_value_import_2 =\
            pd.DataFrame(import_volume_slice.\
            loc[import_volume_slice["reporter_name"] == "Afghanistan",:].\
            groupby("product_group")["trade_value"].\
            sum().\
            sort_values(ascending = False)).\
            reset_index(level=0)

one_country_total_value_import_2.columns = ["Группа товаров", "Объем импорта, в ден. выражении"]
one_country_total_value_import_2["Группа товаров"] = one_country_total_value_import_2["Группа товаров"].str.capitalize()
one_country_total_value_import_2.head()

,Группа товаров,"Объем импорта, в ден. выражении"
0,Зерно и продукты перемола,697338879
1,Готовое продовольствие,355033489
2,Химические вещества и материалы,340226369
3,Текстильные материалы и изделия,334519589
4,Приборы,261986141


#### Рост импорта за 5 лет в Афганистане, Алжире и Албании
#### Так как в выборку попали страны с незначительным количеством категорий иморта стабильно представленных на продолжении длительного промежутка времени - не представляется возможным рассчитать динамку в течение 5 лет

Визуально ни один ряд из напечатанной таблицы не повторяется - необходим больший объем данных

In [309]:
import datetime
past_5_years = [int(datetime.datetime.now().year) - 1 - i for i in range(0, 5)]

five_year_import_volume_slice =\
import_volume_slice.loc[import_volume_slice["year"].isin(past_5_years), ["year", "reporter_name", 
                                                                         "product_group", "trade_value"]]

annual_total = pd.DataFrame(five_year_import_volume_slice.groupby(["reporter_name", 
                                                                   "product_group", 
                                                                   "year"])["trade_value"].sum()).reset_index()

annual_total.sort_values(["reporter_name", "product_group", "year"])

,reporter_name,product_group,year,trade_value
0,Afghanistan,автотехника,2016,64392116
1,Afghanistan,бумага и картон,2016,72663237
2,Afghanistan,бытовая техника,2016,11289598
3,Afghanistan,готовое продовольствие,2016,355033489
4,Afghanistan,готовые химические продукты,2016,100834046
5,Afghanistan,зерно и продукты перемола,2016,697338879
6,Afghanistan,изделия из драгоценных металлов и камней,2016,14640826
7,Afghanistan,корма,2016,986539
8,Afghanistan,металлоизделия,2016,65313719
9,Afghanistan,напитки,2016,25502954


#### Коэффициент Херфиндаля-Хиршмана, 2016
Так как только в отношении Афганистана представлены данные 2016 года - только эта страна учтена в анализе

In [347]:
index_slice = read_final.loc[read_final.year == 2016,["year", "reporter_name", "partner_name",
                                                                         "product_group", "trade_value"]]
index_slice_group = index_slice.groupby(["reporter_name", "product_group"])["trade_value"].sum().reset_index()

aggregate_import_index = index_slice.groupby(["reporter_name", "product_group", 
                                              "partner_name"])["trade_value"].sum().reset_index()

final_table = aggregate_import_index.merge(index_slice_group, on = ["reporter_name", "product_group"], how = "left")
final_table["prop"] = (final_table["trade_value_y"] / final_table["trade_value_x"])
final_table["index"] = final_table["prop"]**2
final_table.groupby(["reporter_name", "product_group"])["index"].sum().sort_values(ascending = False).reset_index()



,reporter_name,product_group,index
0,Afghanistan,текстильные материалы и изделия,1.665861e+12
1,Afghanistan,приборы,1.148251e+12
2,Afghanistan,специальное отраслевое оборудование,1.116414e+12
3,Afghanistan,металлоизделия,1.101119e+12
4,Afghanistan,химические вещества и материалы,8.762222e+11
5,Afghanistan,автотехника,6.687570e+11
6,Afghanistan,зерно и продукты перемола,4.532329e+11
7,Afghanistan,плодоовощная продукция,3.287146e+11
8,Afghanistan,разные промышленные товары,2.878697e+11
9,Afghanistan,готовое продовольствие,1.312706e+11
